In [ ]:
!git clone https://github.com/Cld338/Aspect-Term-Extraction-and-Analysis-Custom ATE
!pip install transformers

Cloning into 'ATE'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 39 (delta 12), reused 33 (delta 8), pack-reused 0
Receiving objects: 100% (39/39), 737.04 KiB | 6.82 MiB/s, done.
Resolving deltas: 100% (12/12), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.9 MB/s eta 0:00:00


In [ ]:
epoch = 10
ATE_file = f"bert_ATE_epoch{epoch}.pkl"
ABSA_file = f"bert_ABSA_epoch{epoch}.pkl"
file_save_path = "/content/gdrive/MyDrive/2023_sw_inje_proj/bert-base-multilingual-uncased"

# import

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
from ATE.model.bert import bert_ATE, bert_ABSA
from ATE.data.dataset import dataset_ATM, dataset_ABSA

In [ ]:
from torch.utils.data import DataLoader, ConcatDataset
from transformers import BertTokenizer
import torch
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import time
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# Settings

In [ ]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
pretrain_model_name = "bert-base-multilingual-uncased"
tokenizer = BertTokenizer.from_pretrained(pretrain_model_name)
lr = 2e-5
model_ATE = bert_ATE(pretrain_model_name).to(DEVICE)
optimizer_ATE = torch.optim.Adam(model_ATE.parameters(), lr=lr)
model_ABSA = bert_ABSA(pretrain_model_name).to(DEVICE)
optimizer_ABSA = torch.optim.Adam(model_ABSA.parameters(), lr=lr)

In [ ]:


def evl_time(t):
    min, sec= divmod(t, 60)
    hr, min = divmod(min, 60)
    return int(hr), int(min), int(sec)

def load_model(model, path):
    model.load_state_dict(torch.load(path), strict=False)
    return model

def save_model(model, name):
    torch.save(model.state_dict(), name)

# Acpect Term Extraction

In [ ]:
laptops_train_ds = dataset_ATM(pd.read_csv("ATE/data/laptops_train.csv"), tokenizer)
laptops_test_ds = dataset_ATM(pd.read_csv("ATE/data/laptops_test.csv"), tokenizer)
restaurants_train_ds = dataset_ATM(pd.read_csv("ATE/data/restaurants_train.csv"), tokenizer)
restaurants_test_ds = dataset_ATM(pd.read_csv("ATE/data/restaurants_test.csv"), tokenizer)
twitter_train_ds = dataset_ATM(pd.read_csv("ATE/data/twitter_train.csv"), tokenizer)
twitter_test_ds = dataset_ATM(pd.read_csv("ATE/data/twitter_test.csv"), tokenizer)
data_path = "/content/gdrive/MyDrive/2023_sw_inje_proj/data"
custom_train_ds = dataset_ATM(pd.read_csv(f"{data_path}/processed_nikluge-sa-2022-train.csv"), tokenizer)
custom_test_ds = dataset_ATM(pd.read_csv(f"{data_path}/processed_nikluge-sa-2022-dev.csv"), tokenizer)

In [ ]:
# w,x,y,z = laptops_train_ds.__getitem__(121)
# print(w)
# print(x)
# print(x.size())
# print(y)
# print(y.size())
# print(z)
# print(z.size())

In [ ]:

# train_ds = ConcatDataset([laptops_train_ds, restaurants_train_ds, twitter_train_ds])
# test_ds = ConcatDataset([laptops_test_ds, restaurants_test_ds, twitter_test_ds])
train_ds = ConcatDataset([custom_train_ds])
test_ds = ConcatDataset([custom_train_ds])

In [ ]:
def create_mini_batch(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors = pad_sequence(ids_tensors, batch_first=True)

    tags_tensors = [s[2] for s in samples]
    tags_tensors = pad_sequence(tags_tensors, batch_first=True)

    pols_tensors = [s[3] for s in samples]
    pols_tensors = pad_sequence(pols_tensors, batch_first=True)

    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1)

    return ids_tensors, tags_tensors, pols_tensors, masks_tensors

In [ ]:
train_loader = DataLoader(train_ds, batch_size=5, collate_fn=create_mini_batch, shuffle = True)
test_loader = DataLoader(test_ds, batch_size=50, collate_fn=create_mini_batch, shuffle = True)

In [ ]:
# for batch in train_loader:
#     w,x,y,z = batch
#     print(w)
#     print(w.size())
#     print(x)
#     print(x.size())
#     print(y)
#     print(y.size())
#     print(z)
#     print(z.size())
#     break

In [ ]:
def train_model_ATE(loader, epochs):
    all_data = len(loader)
    for epoch in range(epochs):
        finish_data = 0
        losses = []
        current_times = []
        correct_predictions = 0

        for data in loader:
            t0 = time.time()
            ids_tensors, tags_tensors, _,  masks_tensors = data
            ids_tensors = ids_tensors.to(DEVICE)
            tags_tensors = tags_tensors.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)

            loss = model_ATE(ids_tensors=ids_tensors, tags_tensors=tags_tensors, masks_tensors=masks_tensors)
            losses.append(loss.item())
            loss.backward()
            optimizer_ATE.step()
            optimizer_ATE.zero_grad()

            finish_data += 1
            current_times.append(round(time.time()-t0,3))
            current = np.mean(current_times)
            hr, min, sec = evl_time(current*(all_data-finish_data) + current*all_data*(epochs-epoch-1))
            print('epoch:', epoch, " batch:", finish_data, "/" , all_data, " loss:", np.mean(losses), " hr:", hr, " min:", min," sec:", sec)

        save_model(model_ATE, f'{file_save_path}/{ATE_file}')

def test_model_ATE(loader):
    pred = []
    trueth = []
    with torch.no_grad():
        for data in loader:

            ids_tensors, tags_tensors, _, masks_tensors = data
            ids_tensors = ids_tensors.to(DEVICE)
            tags_tensors = tags_tensors.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)

            outputs = model_ATE(ids_tensors=ids_tensors, tags_tensors=None, masks_tensors=masks_tensors)

            _, predictions = torch.max(outputs, dim=2)

            pred += list([int(j) for i in predictions for j in i ])
            trueth += list([int(j) for i in tags_tensors for j in i ])

    return trueth, pred



In [ ]:
%time train_model_ATE(train_loader, epoch)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
epoch: 1  batch: 403 / 600  loss: 0.007142125499532733  hr: 0  min: 3  sec: 14
epoch: 1  batch: 404 / 600  loss: 0.007125541901330345  hr: 0  min: 3  sec: 14
epoch: 1  batch: 405 / 600  loss: 0.007147209881360242  hr: 0  min: 3  sec: 14
epoch: 1  batch: 406 / 600  loss: 0.007130466263786308  hr: 0  min: 3  sec: 14
epoch: 1  batch: 407 / 600  loss: 0.007113497748732184  hr: 0  min: 3  sec: 14
epoch: 1  batch: 408 / 600  loss: 0.007114419094861268  hr: 0  min: 3  sec: 14
epoch: 1  batch: 409 / 600  loss: 0.007102206973135018  hr: 0  min: 3  sec: 14
epoch: 1  batch: 410 / 600  loss: 0.007086537886403321  hr: 0  min: 3  sec: 14
epoch: 1  batch: 411 / 600  loss: 0.007084351766148987  hr: 0  min: 3  sec: 14
epoch: 1  batch: 412 / 600  loss: 0.0070987478164355605  hr: 0  min: 3  sec: 14
epoch: 1  batch: 413 / 600  loss: 0.007082739231892396  hr: 0  min: 3  sec: 14
epoch: 1  batch: 414 / 600  loss: 0.007067761272935238  hr: 0  min: 3  sec: 14
epoch: 1  batch

In [ ]:
model_ATE = load_model(model_ATE, f'{file_save_path}/{ATE_file}')

In [ ]:
%time x, y = test_model_ATE(test_loader)
print(classification_report(x, y, target_names=[str(i) for i in range(3)]))
with open(f"{file_save_path}/ATE_epoch{epoch}_report.txt", "w") as file:
  file.write(classification_report(x, y, target_names=[str(i) for i in range(3)]))

CPU times: user 18.8 s, sys: 178 ms, total: 19 s
Wall time: 18.9 s
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    328132
           1       0.98      0.96      0.97      5179
           2       0.97      0.99      0.98     10089

    accuracy                           1.00    343400
   macro avg       0.98      0.98      0.98    343400
weighted avg       1.00      1.00      1.00    343400



# Aspect Based Sentiment Analysis

In [ ]:
laptops_train_ds = dataset_ABSA(pd.read_csv("ATE/data/laptops_train.csv"), tokenizer)
laptops_test_ds = dataset_ABSA(pd.read_csv("ATE/data/laptops_test.csv"), tokenizer)
restaurants_train_ds = dataset_ABSA(pd.read_csv("ATE/data/restaurants_train.csv"), tokenizer)
restaurants_test_ds = dataset_ABSA(pd.read_csv("ATE/data/restaurants_test.csv"), tokenizer)
twitter_train_ds = dataset_ABSA(pd.read_csv("ATE/data/twitter_train.csv"), tokenizer)
twitter_test_ds = dataset_ABSA(pd.read_csv("ATE/data/twitter_test.csv"), tokenizer)

In [ ]:
w,x,y,z = laptops_train_ds.__getitem__(121)
print(w)
print(len(w))
print(x)
print(len(x))
print(y)
print(len(y))
print(z)

['[cls]', 'the', 'battery', 'life', 'seems', 'to', 'be', 'very', 'good', ',', 'and', 'have', 'had', 'no', 'issues', 'with', 'it', '.', '[sep]', 'battery', 'life']
21
tensor([  100, 10103, 34794, 10287, 32681, 10114, 10346, 12495, 12050,   117,
        10110, 10574, 10407, 10181, 17156, 10171, 10197,   119,   100, 34794,
        10287])
21
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1])
21
tensor(2)


In [ ]:
def create_mini_batch2(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors = pad_sequence(ids_tensors, batch_first=True)

    segments_tensors = [s[2] for s in samples]
    segments_tensors = pad_sequence(segments_tensors, batch_first=True)

    label_ids = torch.stack([s[3] for s in samples])

    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1)

    return ids_tensors, segments_tensors, masks_tensors, label_ids

In [ ]:
train_ds = ConcatDataset([laptops_train_ds, restaurants_train_ds, twitter_train_ds])
test_ds = ConcatDataset([laptops_test_ds, restaurants_test_ds, twitter_test_ds])

train_loader = DataLoader(train_ds, batch_size=4, collate_fn=create_mini_batch2, shuffle = True)
test_loader = DataLoader(test_ds, batch_size=50, collate_fn=create_mini_batch2, shuffle = True)

In [ ]:
# for batch in train_loader:
#     w,x,y,z = batch
#     print(w)
#     print(w.size())
#     print(x)
#     print(x.size())
#     print(y)
#     print(y.size())
#     print(z)
#     print(z.size())
#     break

In [ ]:
def train_model_ABSA(loader, epochs):
    all_data = len(loader)
    for epoch in range(epochs):
        finish_data = 0
        losses = []
        current_times = []
        correct_predictions = 0

        for data in loader:
            t0 = time.time()
            ids_tensors, segments_tensors, masks_tensors, label_ids = data
            ids_tensors = ids_tensors.to(DEVICE)
            segments_tensors = segments_tensors.to(DEVICE)
            label_ids = label_ids.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)

            loss = model_ABSA(ids_tensors=ids_tensors, lable_tensors=label_ids, masks_tensors=masks_tensors, segments_tensors=segments_tensors)
            losses.append(loss.item())
            loss.backward()
            optimizer_ABSA.step()
            optimizer_ABSA.zero_grad()

            finish_data += 1
            current_times.append(round(time.time()-t0,3))
            current = np.mean(current_times)
            hr, min, sec = evl_time(current*(all_data-finish_data) + current*all_data*(epochs-epoch-1))
            print('epoch:', epoch, " batch:", finish_data, "/" , all_data, " loss:", np.mean(losses), " hr:", hr, " min:", min," sec:", sec)

        save_model(model_ABSA, f'{file_save_path}/{ABSA_file}')

def test_model_ABSA(loader):
    pred = []
    trueth = []
    with torch.no_grad():
        for data in loader:

            ids_tensors, segments_tensors, masks_tensors, label_ids = data
            ids_tensors = ids_tensors.to(DEVICE)
            segments_tensors = segments_tensors.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)

            outputs = model_ABSA(ids_tensors, None, masks_tensors=masks_tensors, segments_tensors=segments_tensors)

            _, predictions = torch.max(outputs, dim=1)

            pred += list([int(i) for i in predictions])
            trueth += list([int(i) for i in label_ids])

    return trueth, pred



In [ ]:
%time train_model_ABSA(train_loader, epoch)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
epoch: 8  batch: 1091 / 3044  loss: 0.06860010555186023  hr: 0  min: 3  sec: 6
epoch: 8  batch: 1092 / 3044  loss: 0.06857506279744491  hr: 0  min: 3  sec: 6
epoch: 8  batch: 1093 / 3044  loss: 0.0685367832029126  hr: 0  min: 3  sec: 6
epoch: 8  batch: 1094 / 3044  loss: 0.06848063963919561  hr: 0  min: 3  sec: 6
epoch: 8  batch: 1095 / 3044  loss: 0.06864051434825935  hr: 0  min: 3  sec: 6
epoch: 8  batch: 1096 / 3044  loss: 0.06857969420488302  hr: 0  min: 3  sec: 6
epoch: 8  batch: 1097 / 3044  loss: 0.06853678187454304  hr: 0  min: 3  sec: 6
epoch: 8  batch: 1098 / 3044  loss: 0.06847588915546647  hr: 0  min: 3  sec: 6
epoch: 8  batch: 1099 / 3044  loss: 0.06841735692895623  hr: 0  min: 3  sec: 6
epoch: 8  batch: 1100 / 3044  loss: 0.06836866988609969  hr: 0  min: 3  sec: 6
epoch: 8  batch: 1101 / 3044  loss: 0.0683098377101208  hr: 0  min: 3  sec: 6
epoch: 8  batch: 1102 / 3044  loss: 0.06828343122500331  hr: 0  min: 3  sec: 6
epoch: 8  batch: 1

In [ ]:
model_ABSA = load_model(model_ABSA, f'{file_save_path}/{ABSA_file}')

In [ ]:
%time x, y = test_model_ABSA(test_loader)
print(classification_report(x, y, target_names=[str(i) for i in range(3)]))
with open(f"{file_save_path}/ABSA_epoch{epoch}_report.txt", "w") as file:
  file.write(classification_report(x, y, target_names=[str(i) for i in range(3)]))

CPU times: user 5.6 s, sys: 22.8 ms, total: 5.62 s
Wall time: 5.6 s
              precision    recall  f1-score   support

           0       0.67      0.69      0.68       497
           1       0.70      0.61      0.65       710
           2       0.83      0.88      0.85      1239

    accuracy                           0.76      2446
   macro avg       0.73      0.73      0.73      2446
weighted avg       0.76      0.76      0.76      2446



# ATE + ABSA

In [ ]:
def predict_model_ABSA(sentence, aspect, tokenizer):
    t1 = tokenizer.tokenize(sentence)
    t2 = tokenizer.tokenize(aspect)

    word_pieces = ['[cls]']
    word_pieces += t1
    word_pieces += ['[sep]']
    word_pieces += t2

    segment_tensor = [0] + [0]*len(t1) + [0] + [1]*len(t2)

    ids = tokenizer.convert_tokens_to_ids(word_pieces)
    input_tensor = torch.tensor([ids]).to(DEVICE)
    segment_tensor = torch.tensor(segment_tensor).to(DEVICE)

    with torch.no_grad():
        outputs = model_ABSA(input_tensor, None, None, segments_tensors=segment_tensor)
        _, predictions = torch.max(outputs, dim=1)

    return word_pieces, predictions, outputs

def predict_model_ATE(sentence, tokenizer):
    word_pieces = []
    tokens = tokenizer.tokenize(sentence)
    word_pieces += tokens

    ids = tokenizer.convert_tokens_to_ids(word_pieces)
    input_tensor = torch.tensor([ids]).to(DEVICE)

    with torch.no_grad():
        outputs = model_ATE(input_tensor, None, None)
        _, predictions = torch.max(outputs, dim=2)
    predictions = predictions[0].tolist()

    return word_pieces, predictions, outputs

def ATE_ABSA(text):
    terms = []
    word = ""
    x, y, z = predict_model_ATE(text, tokenizer)
    for i in range(len(y)):
        if y[i] == 1:
            if len(word) != 0:
                terms.append(word.replace("##","").replace(" ", ""))
            word = x[i]
        if y[i] == 2:
            word += (" " + x[i])


    if len(word) != 0:
            terms.append(word.replace("##",""))

    print("tokens:", x)
    print("ATE:", terms)

    if len(terms) != 0:
        for i in terms:
            _, c, p = predict_model_ABSA(text, i, tokenizer)
            print("term:", [i], "class:", [int(c)], "ABSA:", [float(p[0][0]), float(p[0][1]), float(p[0][2])])


In [ ]:
model_ABSA = load_model(model_ABSA, f'{file_save_path}/{ABSA_file}')
model_ATE = load_model(model_ATE, f'{file_save_path}/{ATE_file}')

In [ ]:
text = "고밀도 폼 소재라..빈틈이 없고 커버만 관리하면 되고, 항균폼이 내재되어 99.9% 항균이 가능해요~"
text = re.sub('[^가-힣a-z.]', ' ', text)
ATE_ABSA(text)

tokens: ['고', '##미', '##ᆯ', '##도', 'ᄑ', '##ᅩᆷ', 'ᄉ', '##ᅩ', '##재', '##라', '.', '.', 'ᄇ', '##ᅵᆫ', '##트', '##ᆷ이', 'ᄋ', '##ᅥ', '##ᆹ', '##고', 'ᄏ', '##ᅥ', '##버', '##만', '과', '##ᆫ리', '##하면', '되고', ',', 'ᄒ', '##ᅡᆼ', '##균', '##포', '##ᆷ이', '내', '##재', '##되어', '99', '.', '9', '%', 'ᄒ', '##ᅡᆼ', '##균', '##이', '가', '##능', '##해', '##요', '~']
ATE: []


In [ ]:
text = "제품만족도 94% 제품이라 믿고 사용해본 #산다화클렌징오일"
text = re.sub('[^가-힣a-z.]', ' ', text)
ATE_ABSA(text)

tokens: ['제', '##품', '##만', '##족', '##도', '94', '%', '제', '##품', '##이', '##라', 'ᄆ', '##ᅵ', '##ᆮ고', 'ᄉ', '##ᅡ', '##용', '##해', '##본', '#', 'ᄉ', '##ᅡᆫ', '##다', '##화', '##크', '##ᆯ', '##레', '##ᆫ', '##지', '##ᆼ', '##오', '##일']
ATE: ['ᄉ', 'ᅡᆫ', '다', '화', '크 ᆯ 레 ᆫ 지 ᆼ 오 일']
term: ['ᄉ'] class: [1] ABSA: [-0.5015776753425598, 0.565009593963623, -0.01903115026652813]
term: ['ᅡᆫ'] class: [2] ABSA: [-0.050442956387996674, -0.13797792792320251, 0.11214940994977951]
term: ['다'] class: [1] ABSA: [0.058123260736465454, 0.0640985518693924, -0.15378838777542114]
term: ['화'] class: [0] ABSA: [0.23215143382549286, 0.06062309816479683, -0.3739544749259949]
term: ['크 ᆯ 레 ᆫ 지 ᆼ 오 일'] class: [1] ABSA: [-0.6520639061927795, 0.7885296940803528, -0.05594963580369949]


In [ ]:
text = "박민 신임 한국방송(KBS) 사장 취임 첫날인 13일, 간판 뉴스인 ‘뉴스9’ 앵커로 낙점된 박장범 기자가 첫 방송에서 기존 뉴스가 공영방송의 정체성을 흔들고 정파성 논란을 불러일으켰다고 평가했다."
ATE_ABSA(text)

tokens: ['ᄇ', '##ᅡᆨ', '##민', '시', '##ᆫ이', '##ᆷ', '한국', '##방송', '(', 'kbs', ')', 'ᄉ', '##ᅡ', '##장', '취임', '첫', '##날', '##인', '13일', ',', '가', '##ᆫ', '##판', 'ᄂ', '##ᅲ스', '##인', '[UNK]', 'ᄂ', '##ᅲ스', '##9', '[UNK]', 'ᄋ', '##ᅢ', '##ᆼ', '##ᄏ', '##ᅥ', '##로', 'ᄂ', '##ᅡᆨ', '##점', '##된', 'ᄇ', '##ᅡᆨ', '##장', '##버', '##ᆷ', '기자', '##가', '첫', '방송', '##에서', 'ᄀ', '##ᅵ', '##존', 'ᄂ', '##ᅲ스', '##가', '고', '##ᆼ', '##영', '##방송', '##의', '정', '##체', '##성을', 'ᄒ', '##ᅳᆫ', '##들', '##고', '정', '##파', '##성', 'ᄂ', '##ᅩᆫ', '##란', '##을', 'ᄇ', '##ᅮᆯ', '##러', '##일', '##ᄋ', '##ᅳ', '##켰다', '##고', 'ᄑ', '##ᅧᆼ', '##가', '##했다', '.']
ATE: ['한국 방송 ( kbs ) ᄉ ᅡ 장']
term: ['한국 방송 ( kbs ) ᄉ ᅡ 장'] class: [1] ABSA: [-1.6649800539016724, 5.402393817901611, -2.832306385040283]


In [ ]:
text = """13일 박 앵커는 ‘뉴스9’ 오프닝 멘트에서 “KBS는 객관적이고 균형 잡힌 뉴스를 통해 정확하고 편견 없는 뉴스를 시청자들에게 전달하는 것을 공영방송의 가장 중요한 책무로 규정하고 있다”며 “그동안 공영방송의 정체성을 흔들었던 정파성 논란을 극복하고 앞으로 공영성을 최우선 가치로 하는 뉴스 프로그램을 방송해 시청자들의 신뢰를 회복하기 위해서 노력하겠다”고 말했다."""
ATE_ABSA(text)

tokens: ['13일', 'ᄇ', '##ᅡᆨ', 'ᄋ', '##ᅢ', '##ᆼ', '##ᄏ', '##ᅥ', '##는', '[UNK]', 'ᄂ', '##ᅲ스', '##9', '[UNK]', 'ᄋ', '##ᅩ', '##프', '##니', '##ᆼ', 'ᄆ', '##ᅦᆫ', '##트', '##에서', '[UNK]', 'kbs', '##는', 'ᄀ', '##ᅢᆨ', '##관', '##적', '##이고', 'ᄀ', '##ᅲ', '##ᆫ', '##형', 'ᄌ', '##ᅡ', '##ᆸ', '##히', '##ᆫ', 'ᄂ', '##ᅲ스', '##를', '통해', '정', '##화', '##ᆨ', '##하고', 'ᄑ', '##ᅧᆫ', '##겨', '##ᆫ', '없는', 'ᄂ', '##ᅲ스', '##를', '시', '##청', '##자', '##들에게', '전', '##달', '##하는', '것을', '고', '##ᆼ', '##영', '##방송', '##의', '가장', '중요한', 'ᄎ', '##ᅢᆨ', '##무', '##로', 'ᄀ', '##ᅲ', '##정', '##하고', '있다', '[UNK]', 'ᄆ', '##ᅧ', '[UNK]', '그', '##동안', '고', '##ᆼ', '##영', '##방송', '##의', '정', '##체', '##성을', 'ᄒ', '##ᅳᆫ', '##들어', '##ᆻ던', '정', '##파', '##성', 'ᄂ', '##ᅩᆫ', '##란', '##을', '그', '##ᆨ', '##복', '##하고', 'ᄋ', '##ᅡ', '##ᇁ', '##으로', '고', '##ᆼ', '##영', '##성을', 'ᄎ', '##ᅬ', '##우', '##선', '가', '##치', '##로', '하는', 'ᄂ'

# Cyberpunk 2077 - Xbox One

https://www.amazon.com/-/zh_TW/Cyberpunk-2077-Xbox-One/product-reviews/B07DJW4WZC/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=2

In [ ]:
text = "Spent 5 hours downloading updates."
ATE_ABSA(text)

tokens: ['spent', '5', 'hours', 'download', '##ing', 'updates', '.']
ATE: []


In [ ]:
text = "Install is buggy, so after downloading a day one patch that's nearly 3 times the size of the game, it glitched on the CDs and had to reinstall the game from scratch."
ATE_ABSA(text)

tokens: ['instal', '##l', 'is', 'bug', '##gy', ',', 'so', 'after', 'download', '##ing', 'a', 'day', 'one', 'patch', 'that', "'", 's', 'nearly', '3', 'times', 'the', 'size', 'of', 'the', 'game', ',', 'it', 'gli', '##tch', '##ed', 'on', 'the', 'cds', 'and', 'had', 'to', 'rein', '##stal', '##l', 'the', 'game', 'from', 'scratch', '.']
ATE: []


In [ ]:
text = "Cyberpunk 2077 freezes constantly, frame rates are terrible, and it's extremely frustrating to try to play."
ATE_ABSA(text)

tokens: ['cyber', '##pun', '##k', '207', '##7', 'free', '##zes', 'constantly', ',', 'frame', 'rates', 'are', 'terrible', ',', 'and', 'it', "'", 's', 'extremely', 'fru', '##strat', '##ing', 'to', 'try', 'to', 'play', '.']
ATE: [' 2077']
term: [' 2077'] class: [1] ABSA: [-0.27309536933898926, 2.6759541034698486, -2.1472830772399902]


In [ ]:
text = "Cyberpunk 2077 is completely unplayable on xbox one. They should have never released this for current gen."
ATE_ABSA(text)

tokens: ['cyber', '##pun', '##k', '207', '##7', 'is', 'completely', 'un', '##play', '##able', 'on', 'xbox', 'one', '.', 'they', 'should', 'have', 'never', 'released', 'this', 'for', 'current', 'gen', '.']
ATE: [' 207']
term: [' 207'] class: [1] ABSA: [0.5483392477035522, 2.896608591079712, -3.184424638748169]


In [ ]:
text = "It’s just a cash grab, the game crashes constantly, runs at like 20 fps, half the environment and characters only load when you’re three feet away from them. Unless you’re in a small space the game looks awful. The worst game i’ve ever played in years visually. It looks worse than later xbox 360 games."
ATE_ABSA(text)

tokens: ['it', '[UNK]', 's', 'just', 'a', 'cash', 'grab', ',', 'the', 'game', 'crash', '##es', 'constantly', ',', 'runs', 'at', 'like', '20', 'f', '##ps', ',', 'half', 'the', 'environment', 'and', 'characters', 'only', 'load', 'when', 'you', '[UNK]', 're', 'three', 'feet', 'away', 'from', 'them', '.', 'unless', 'you', '[UNK]', 're', 'in', 'a', 'small', 'space', 'the', 'game', 'looks', 'aw', '##ful', '.', 'the', 'worst', 'game', 'i', '[UNK]', 've', 'ever', 'played', 'in', 'years', 'visual', '##ly', '.', 'it', 'looks', 'worse', 'than', 'later', 'xbox', '360', 'games', '.']
ATE: [' 360 games']
term: [' 360 games'] class: [1] ABSA: [1.2824668884277344, 1.6601512432098389, -2.910120725631714]


In [ ]:
text = "CD Projekt Red should have just abandoned the current gen consoles instead of cheating people out of their money."
ATE_ABSA(text)

tokens: ['cd', 'projekt', 'red', 'should', 'have', 'just', 'abandoned', 'the', 'current', 'gen', 'console', '##s', 'instead', 'of', 'che', '##ating', 'people', 'out', 'of', 'their', 'money', '.']
ATE: ['gen consoles']
term: ['gen consoles'] class: [0] ABSA: [3.436600923538208, -0.5347150564193726, -3.2178144454956055]
